In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import pixiedust
from pathlib import Path

Pixiedust database opened successfully


In [2]:
# HERE IS WHERE I CHANGE VIDEO LOCATION AND TRACKER TYPE FOR SAVING AND LOADING DATA:
# Specific video location
Video = "single_country"

# NOW EVERYTHING ELSE GETS DONE AUTOMATICALLY

# Root directory of the project
ROOT_DIR = os.path.abspath("C:/Users/ddefr/OneDrive/Documents/Skripsie/Mask_RCNN")

# Directory for all videos
all_vids = Path("C:/Users/ddefr/OneDrive/Documents/skripsie_videos")

#Specific video dir
video_dir = all_vids / Video

# Measurements to load:
save_measurements = video_dir / "measurements_1.npy"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
%matplotlib inline

Using TensorFlow backend.


In [3]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('', "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [4]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MAX_INSTANCES = 1
    DETECTION_MIN_CONFIDENCE = 0.9
    
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE           

In [5]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_coco.h5', by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [6]:
# COCO Class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [7]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image


def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image


In [8]:
#Set Paths
vid_path = video_dir / "original.mp4"
proccessed_image_path = video_dir / "mask_pics/"
#Start Capture + get number of frames
capture = cv2.VideoCapture(str(vid_path))
tot_frames = capture.get(7)

#initialize variables
frame_count = 0
frames = []
tot_results = [] #rename to measurements
fil_ids = []
fil_masks = []
fil_score = []
fil_rois = []
fil_results = []
properties = []
measurements = []
while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    if len(frames) == config.BATCH_SIZE:
        results = model.detect(frames, verbose=0)
        for x in range (len(results[0]['class_ids'])):
            #if (results[0]['class_ids'][x] == 3):
            fil_ids.append(results[0]['class_ids'][x])
            fil_masks.append(results[0]['masks'][:,:,x])
            fil_score.append(results[0]['scores'][x])
            fil_rois.append(results[0]['rois'][x])
        fil_ids = np.array(fil_ids)
        fil_score = np.array(fil_score)
        fil_rois = np.array(fil_rois)
        if len(fil_ids) > 0:
            fil_masks = np.dstack(fil_masks)
        fil_results.append({'rois': fil_rois, 'masks': fil_masks, 'class_ids': fil_ids, 'scores': fil_score})
        tot_results.insert(frame_count,fil_results)
        
        for i, item in enumerate(zip(frames, fil_results)):
            frame = item[0]
            r = item[1]
            frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
            cv2.imshow('frame', frame)
            name = '{0}.jpg'.format(frame_count + i - config.BATCH_SIZE)
            name = os.path.join(str(proccessed_image_path), name)
            cv2.imwrite(name, frame)
        # Clear the frames array to start the next batch
        frames = []
        fil_ids = []
        fil_masks = []
        fil_score = []
        fil_rois = []
        fil_results = []
        
        #If no measurements then skip
        if len(tot_results[0][0]['class_ids']) > 0:
            for n in range (tot_results[0][0]['masks'].shape[2]):
                label_img = skimage.morphology.label(tot_results[0][0]['masks'].T[n].T)
                props = skimage.measure.regionprops(label_img)
                xc = int(round(props[0]['Centroid'][1]))
                yc = int(round(props[0]['Centroid'][0]))
                area = int(round(props[0]['area']))
                properties.append([xc, 1080-yc, area])        
        else:
            properties = [None,None,None]
        measurements.append(properties)
        
        properties =[]
        tot_results = []
# save numpy array as npy file
from numpy import asarray
from numpy import save
# save to npy file
save(save_measurements, measurements)


NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
NO INSTANCES TO